In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [5]:
# Load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [6]:
# Drop irrevelant features

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [9]:
# Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [13]:
# Onehot encode geographic column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [18]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [16]:
geo_encoder_columns = onehot_encoder_geo.get_feature_names_out(['Geography'])

In [20]:
geo_encoded_df= pd.DataFrame(geo_encoder.toarray(), columns=geo_encoder_columns)

In [21]:
#Combine one hot encoded columns to the original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [22]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [23]:
## Save the LabelEncoder and the OneHotEncoders

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open("onehot_encodee_geo.pkl", 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)



In [25]:
# Divide the dataset into the independent and dependent features

x = data.drop('Exited', axis=1)
y = data['Exited']

#Split the data in training and testing sets

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [27]:
with open("scaler.pkl", 'wb') as file:
    pickle.dump(scaler, file)

ANN IMPLEMENTATION

In [28]:
import tensorflow as tf

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [32]:
## Build our Model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32, activation='relu'),  ## HL2
    Dense(1, activation='sigmoid') ## Output layer
])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [36]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [37]:
## compile the model
model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [43]:
# Set up Early STopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [44]:
## Train the model

history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 10s 39ms/step - loss: 0.3333 - accuracy: 0.8611 - val_loss: 0.3478 - val_accuracy: 0.8570
Epoch 2/100
250/250 [==============================] - 3s 12ms/step - loss: 0.3281 - accuracy: 0.8640 - val_loss: 0.3418 - val_accuracy: 0.8660
Epoch 3/100
250/250 [==============================] - 3s 13ms/step - loss: 0.3282 - accuracy: 0.8652 - val_loss: 0.3533 - val_accuracy: 0.8540
Epoch 4/100
250/250 [==============================] - 3s 11ms/step - loss: 0.3266 - accuracy: 0.8634 - val_loss: 0.3541 - val_accuracy: 0.8600
Epoch 5/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3227 - accuracy: 0.8665 - val_loss: 0.3418 - val_accuracy: 0.8645
Epoch 6/100
250/250 [==============================] - 3s 10ms/step - loss: 0.3222 - accuracy: 0.8669 - val_loss: 0.3556 - val_accuracy: 0.8590
Epoch 7/100
250/250 [==============================] - 2s 8ms/step - loss: 0.3183 - accuracy: 0.8689 - val_loss: 0.3569 - val_accuracy: 

In [45]:
model.save('model.h5')

In [46]:
# Load Tensorboard Extension

%load_ext tensorboard

In [52]:
!kill 23808

kill: 23808: No such process


In [57]:
tensorboard --logdir=logs --port=6060


Reusing TensorBoard on port 6060 (pid 26840), started 0:02:26 ago. (Use '!kill 26840' to kill it.)

In [ ]:
## Load the pickle file